# 再谈抽象

## 对象：属性+方法  
多态（鸭子类型）：可以对不同的对象执行相同的操作，无需知道对象的类型就可以调用方法   
封装：对外部隐藏有关对象工作原理的细节，无需知道对象的构造就可以使用它   
继承：基于通用列创建专用类   

## 类
一个类的对象是另一个类的对象的子集时，前者就是后者的子类，后者就是前者的超类   
Python中约定俗成使用单数并首字母大写表示类   
要定义子类，只需要加入多出来的方法，或者重写既有的方法   

In [1]:
class Person():
    def set_name(self, name):
        self.name = name
    
    def get_name(self):
        return self.name
    
    def greet(self):
        print('Hello, World! I\'m {}.'.format(self.name))

In [3]:
foo.get_name()
foo.greet()

Hello, World! I'm erwer.


In [4]:
foo.get_name()

'erwer'

In [5]:
foo.greet()

Hello, World! I'm erwer.


### 类里面方法第一个参数关联到它所属的实例，因此无需再调用的时候提供这个参数

## 私有
* 私有属性不能从对象外部访问，只能通过存取器方法（比如set_name,get_name等）访问
* 要想让方法或者属性成为私有的，只需要让其名称以两个下划线__打头即可
* 再类的定义中，对所有以两个下划线打头的名称都进行转换，即在开头加上一个下划线和类名
* 不希望名称被修改，又想发出不要从为外部修改属性或方法的信号，可用一个下划线打头，虽然这只是一种约定，但也有些作用，比如` from module import * `不会导入以一个下划线打头的名称

In [8]:
class Secretive:
    def __inaccessible(self):
        print("Bet you can't see me...")
    
    def accessible(self):
        print("the secret message is:")
        self.__inaccessible()

In [10]:
s = Secretive()
s.__inaccessible()

AttributeError: 'Secretive' object has no attribute '__inaccessible'

In [11]:
s.accessible()

the secret message is:
Bet you can't see me...


In [12]:
s._Secretive__inaccessible()

Bet you can't see me...


## 类的命名空间
一下两种命名方式等价

In [1]:
def foo(x): return x * x
foo = lambda x: x * x

类的所有代码都是在一个特殊的命名空间（类的命名空间）内执行的，而类的所有成员都可以访问这个空间，类定义其实就是要执行的代码段

In [37]:
class MemberCounter:
    '在类作用域内定义了一个变量，所有的成员（实例都可以访问它）'
    numbers = 0
    def init(self):
        MemberCounter.numbers += 1


In [38]:
m1 = MemberCounter()
m1.init()
MemberCounter.numbers

1

In [39]:
m2 = MemberCounter()
m2.init()
MemberCounter.numbers

2

In [40]:
print(m1.numbers, m2.numbers)

2 2


如果在一个实例中给属性members赋值，那么新值被写入m1的一个属性中，遮住了类级变量

In [42]:
m1.numbers = 23
print(m1.numbers, m2.numbers)

23 2


## 指定超类
要指定超类（父类）可在class语句中的**类名后面加上超类名**，并将其用圆括号括起

In [43]:
class Filter:
    def init(self):
        self.blocked = []
    def filter(self, sequence):
        return [x for x in sequence if x not in self.blocked]

In [44]:
class SPAMFilter(Filter):   #SPAMFilter 是Filter的子类
    def init(self):      #重写超类Filter的方法init
        self.blocked = ['SPAM']

In [45]:
s = SPAMFilter()

In [47]:
s.init()
s.filter(['S', 'SPAM','SPAM','Bacon'] )

['S', 'Bacon']

### 要确定一个类是否是另一个类的子类，可使用内置方法issubclass

In [48]:
issubclass(SPAMFilter, Filter)

True

### 如果有一个类，想访问它的基类，可使用特殊属性__bases__

In [49]:
SPAMFilter.__bases__

(__main__.Filter,)

In [50]:
Filter.__bases__

(object,)

### 要确定对象是否特定类的实例，可以用isinstance

In [56]:
s = SPAMFilter()
print(isinstance(s,SPAMFilter), isinstance(s, Filter), isinstance(s, str))
# s是SPAMFilter的直接实例，是Filter的简介实例

True True False


### 要想获知对象属于哪个类，可以使用属性__class__或者type函数

In [57]:
type(s)

__main__.SPAMFilter

In [58]:
s.__class__

__main__.SPAMFilter

## 多个超类
多重继承：一个类可以指定多个超类（父类）
* 如果有多个同名方法，位于前面的类的方法将覆盖后面的类的方法，称为**方法解析顺序（MRO）**

### 检验所需方法是否存在

In [61]:
hasattr(SPAMFilter, 'talk')

False

In [62]:
hasattr(SPAMFilter, 'filter')

True

### 检查方法是否可以调用

In [67]:
callable(getattr(SPAMFilter, 'name', None))

False

In [70]:
callable(getattr(SPAMFilter, 'init', None))

True

### 设置对象属性

In [76]:
setattr(SPAMFilter, 'blocked', 'None')

In [78]:
getattr(SPAMFilter, 'blocked', None)

'None'

## 抽象基类

一般而言，抽象类是不能（至少是不应该）实例化的类，其职责是定义子类应该实现的一组抽象方法

In [83]:
from abc  import ABC, abstractmethod

class Talker(ABC):
    @abstractmethod  #装饰器，用来将方法标记为抽象的--在子类中必须实现的方法
    def talk(self):
        pass

In [84]:
a = Talker()

TypeError: Can't instantiate abstract class Talker with abstract methods talk

In [87]:
class Knigget(Talker):  #因为没有重写方法talk，因此这个类也是抽象的，不能实例化
    pass

In [89]:
a = Knigget()

TypeError: Can't instantiate abstract class Knigget with abstract methods talk

In [91]:
class Knigget(Talker):
    def talk(self):
        print("Hi")
   

In [93]:
a = Knigget()
a.talk()

Hi


## 可以虽然不是派生类，但强行注册为派生类的做法

In [97]:
class Herring:
    def ta(self):
        print("na")

h = Herring()

In [98]:
isinstance(h, Talker)

False

In [100]:
Talker.register(Herring)
isinstance(h, Talker)

True

In [101]:
h.talk()

AttributeError: 'Herring' object has no attribute 'talk'

# 本章完